In [ ]:
!pip install Cython
!pip install https://github.com/Santosh-Gupta/scikit-learn/archive/master.zip

In [48]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

In [51]:
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1]
y = dataset.iloc[:, -1]
x

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [52]:
# Encoding "Gender"
le = LabelEncoder()
x.iloc[:, 2] = le.fit_transform(x.iloc[:, 2])
x

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.80,3,1,0,113931.57
3,699,France,0,39,1,0.00,2,0,0,93826.63
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64
9996,516,France,1,35,10,57369.61,1,1,1,101699.77
9997,709,France,0,36,7,0.00,1,0,1,42085.58
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52


In [53]:
# Enconding "Geography"
x.iloc[:, 1] = le.fit_transform(x[["Geography"]])# As a result, we have: France = 0; Germay = 1; Spain = 2
x

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64
9996,516,0,1,35,10,57369.61,1,1,1,101699.77
9997,709,0,0,36,7,0.00,1,0,1,42085.58
9998,772,1,1,42,3,75075.31,2,1,0,92888.52


In [23]:
# Define training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [24]:
# Feature scaling
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [25]:
# Create our Aritificial Neural Network (ANN)
ann = tf.keras.models.Sequential()

In [26]:
# Add input layer and first hidden layer to ANN
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [27]:
# Add second layer
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [28]:
# Add the output layer
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [31]:
# Compile ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [32]:
# Train ANN on Training set
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.5643 - accuracy: 0.7194
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4591 - accuracy: 0.7977
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4399 - accuracy: 0.8024
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4310 - accuracy: 0.8069
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4244 - accuracy: 0.8117
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4167 - accuracy: 0.8196
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4060 - accuracy: 0.8257
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3938 - accuracy: 0.8379
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3828 - accuracy: 0.8450
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3746 - accura

In [35]:
# Predict if the costumer with the following informations will leave the bank:
#    Greography: France
#    Credit Score: 600
#    Gender: Male
#    Age: 40 years old
#    Tenure: 3 years
#    Balance: $ 60000
#    Have a credit card: yes
#    Is it an active member: yes
#    Estimated salary: $ 50000
print(ann.predict(sc.transform([[0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


In [47]:
# Predict the Test set results
y_pred = ann.predict(x_test)
y_pred = y_pred > 0.5
y_test = np.array(y_test)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [49]:
# Make the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1513   82]
 [ 198  207]]


0.86